In [ ]:
import os
import random
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
from ultralytics import YOLO
from tqdm import tqdm
import time
torch.manual_seed(42)
class TripletDataset(Dataset):
    def __init__(self, data_dir, n_triplets=10000, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        if not os.path.exists(self.data_dir):
            raise ValueError(f"Directory Not Found: {self.data_dir}")

        self.data_imgs = self.load_images()
        print(f'Number of images in data: {len(self.data_imgs)}')

        self.triplets = self.create_triplets(self.data_imgs, n_triplets)
        if not self.triplets:
            raise ValueError("No valid triplets created. Check your data.")

    def load_images(self):
        data_imgs = []

        for subfolder in os.listdir(self.data_dir):
            subfolder_path = os.path.join(self.data_dir, subfolder)
            if not os.path.isdir(subfolder_path):
                continue
            for filename in os.listdir(subfolder_path):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    data_imgs.append(os.path.join(subfolder_path, filename))

        if not data_imgs:
            raise ValueError(f"No valid images found in {self.data_dir}")

        return data_imgs

    def create_triplets(self, data_imgs, n_triplets):
        triplets = []
        data_subfolder_to_images = {}

        for img_path in data_imgs:
            subfolder = os.path.basename(os.path.dirname(img_path))
            if subfolder not in data_subfolder_to_images:
                data_subfolder_to_images[subfolder] = []
            data_subfolder_to_images[subfolder].append(img_path)

        subfolder_list = list(data_subfolder_to_images.keys())
        if len(subfolder_list) < 2:
            raise ValueError("Need at least 2 subfolders (persons) to create triplets")

        skipped_subfolders = 0
        for _ in range(n_triplets):
            anchor_subfolder = random.choice(subfolder_list)
            if len(data_subfolder_to_images[anchor_subfolder]) < 2:
                skipped_subfolders += 1
                continue

            anchor_img, positive_img = random.sample(data_subfolder_to_images[anchor_subfolder], 2)

            negative_subfolder = random.choice([s for s in subfolder_list if s != anchor_subfolder])
            negative_img = random.choice(data_subfolder_to_images[negative_subfolder])

            triplets.append((anchor_img, positive_img, negative_img))

        if not triplets:
            raise ValueError("No valid triplets created. Ensure subfolders have at least 2 images.")
        print(f'Created {len(triplets)} triplets (skipped {skipped_subfolders} subfolders with < 2 images)')
        return triplets

    def __len__(self):
        
        return len(self.triplets)

    def __getitem__(self, idx):

        anchor_path, positive_path, negative_path = self.triplets[idx]
        anchor = Image.open(anchor_path).convert('RGB')
        positive = Image.open(positive_path).convert('RGB')
        negative = Image.open(negative_path).convert('RGB')

        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative

class SiameseNet(nn.Module):
    def __init__(self, feature_extractor, embedding_dim=256):
        super(SiameseNet, self).__init__()
        self.feature_extractor = feature_extractor
        
        with torch.no_grad():
            sample_input = torch.randn(1, 3, 128, 128)
            features = feature_extractor(sample_input)
            feature_dim = features.view(features.size(0), -1).shape[1]
            print(f"Feature dimension: {feature_dim}")
        
        self.dropout = nn.Dropout(0)
        self.projection = nn.Linear(feature_dim, embedding_dim)

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        features = self.dropout(features)
        embedding = self.projection(features)
        embedding = F.normalize(embedding, p=2, dim=1) 
        return embedding

def compute_metrics(model, dataloader, margin, device, desc='Evaluating'):
    model.eval()
    total_loss = 0
    correct_triplets = 0
    total_triplets = 0
    progress_bar = tqdm(dataloader, desc=desc, unit='batch', leave=False)
    
    with torch.no_grad():
        for anchor, positive, negative in progress_bar:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
            anchor_emb = model(anchor)
            positive_emb = model(positive)
            negative_emb = model(negative)
            loss = F.triplet_margin_loss(anchor_emb, positive_emb, negative_emb, margin=margin)
            total_loss += loss.item()
            d_ap = torch.norm(anchor_emb - positive_emb, p=2, dim=1)
            d_an = torch.norm(anchor_emb - negative_emb, p=2, dim=1)
            correct_triplets += (d_ap < d_an).sum().item()
            total_triplets += anchor.size(0)
            progress_bar.set_postfix({'loss': loss.item(), 'd_ap': d_ap.mean().item()})
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct_triplets / total_triplets if total_triplets > 0 else 0
    return avg_loss, accuracy

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO('yolo11n-cls.pt')
feature_extractor = nn.Sequential(*list(model.model.children())[:-1])
siamese_net = SiameseNet(feature_extractor, embedding_dim=256).to(device)

model_save_path = 'best_faces.pt'
if os.path.exists(model_save_path):
    siamese_net.load_state_dict(torch.load(model_save_path))
    print(f"Loaded pre-trained weights from {model_save_path}")
else:
    print(f"No pre-trained model found at {model_save_path}, starting from scratch")

optimizer = torch.optim.Adam(siamese_net.parameters(), lr=0.00001, weight_decay=1e-2) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
margin = 0.6

# Transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    # transforms.RandomHorizontalFlip(p=0.3),
    # transforms.RandomRotation(5),
    # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = TripletDataset(data_dir='data', n_triplets=15000, transform=transform)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size


indices = list(range(total_size))
random.shuffle(indices)
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}")


num_epochs = 50  
best_val_loss = float('inf')
fine_tune_save_path = 'best_face_finetune'
patience = 10
epochs_no_improve = 0

if os.path.exists(model_save_path):
    initial_val_loss, initial_val_acc = compute_metrics(siamese_net, val_loader, margin, device, desc='Initial Validation')
    best_val_loss = initial_val_loss
    print(f"Initial Validation Loss: {initial_val_loss:.4f}, Accuracy: {initial_val_acc:.4f}")

for epoch in range(num_epochs):
    siamese_net.train()
    total_train_loss = 0
    correct_train_triplets = 0
    total_train_triplets = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', unit='batch')
    
    for anchor, positive, negative in progress_bar:
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        optimizer.zero_grad()
        anchor_emb = siamese_net(anchor)
        positive_emb = siamese_net(positive)
        negative_emb = siamese_net(negative)
        loss = F.triplet_margin_loss(anchor_emb, positive_emb, negative_emb, margin=margin)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        d_ap = torch.norm(anchor_emb - positive_emb, p=2, dim=1)
        d_an = torch.norm(anchor_emb - negative_emb, p=2, dim=1)
        correct_train_triplets += (d_ap < d_an).sum().item()
        total_train_triplets += anchor.size(0)
        
        progress_bar.set_postfix({'train_loss': loss.item(), 'd_ap': d_ap.mean().item()})

    # if (epoch %10==0 and epoch >1):
    #     time.sleep(200)

    
    train_loss = total_train_loss / len(train_loader)
    train_acc = correct_train_triplets / total_train_triplets if total_train_triplets > 0 else 0
    
    val_loss, val_acc = compute_metrics(siamese_net, val_loader, margin, device, desc='Validation')
    
    scheduler.step(val_loss)
    
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'  Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    print(f'  Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(siamese_net.state_dict(), fine_tune_save_path)
        print(f'  Model saved to {fine_tune_save_path} (Best Val Loss: {best_val_loss:.4f})')
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f'  No improvement in validation loss for {epochs_no_improve} epoochs')
        if epochs_no_improve >= patience:
            print(f'  Early stopping triggered after {epoch+1} epochs')
            break

test_loss, test_acc = compute_metrics(siamese_net, test_loader, margin, device, desc='Test Evaluation')
print(f'\nTest Evaluation:')
print(f'  Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

Feature dimension: 49152
Loaded pre-trained weights from best_faces.pt
Number of images in data: 996
Created 100000 triplets (skipped 0 subfolders with < 2 images)
Train size: 70000, Val size: 15000, Test size: 15000


Initial Validation Loss: 0.4040, Accuracy: 0.7371


Epoch 1/50 [Train]: 100%|██████████| 137/137 [02:50<00:00,  1.24s/batch, train_loss=0.0225, d_ap=0.514]


Epoch 1/50:
  Train Loss: 0.0755, Train Accuracy: 0.9674
  Val Loss: 0.0267, Val Accuracy: 0.9957
  Model saved to best_face_finetune2.pt (Best Val Loss: 0.0267)


Epoch 2/50 [Train]: 100%|██████████| 137/137 [02:50<00:00,  1.25s/batch, train_loss=0.0107, d_ap=0.487] 


...

Epoch 49/50 [Train]: 100%|██████████| 137/137 [02:50<00:00,  1.25s/batch, train_loss=0, d_ap=0.391]


Epoch 49/50:
  Train Loss: 0.0000, Train Accuracy: 1.0000
  Val Loss: 0.0000, Val Accuracy: 1.0000
  No improvement in validation loss for 6 epoochs


Epoch 50/50 [Train]: 100%|██████████| 137/137 [02:57<00:00,  1.29s/batch, train_loss=0, d_ap=0.385]


Epoch 50/50:
  Train Loss: 0.0000, Train Accuracy: 1.0000
  Val Loss: 0.0000, Val Accuracy: 1.0000
  No improvement in validation loss for 7 epoochs



Test Evaluation:
  Test Loss: 0.0001, Test Accuracy: 1.0000


[2, 5, 6, 7, 8, 10, 13, 14, 17, 23, 27, 30, 34, 35, 36, 37, 38, 39, 43, 46, 47, 51, 52, 53, 54, 58, 59, 60, 62, 63, 65, 66, 69, 70, 71, 72, 75, 76, 79, 81, 82, 83, 84, 85, 86, 89, 90, 91, 93, 95, 99, 103, 104, 105, 106, 107, 110, 112, 113, 115, 117, 118, 119, 120, 121, 127, 129, 130, 132, 134, 135, 139, 141, 142, 143, 144, 145, 147, 148, 150, 152, 154, 155, 156, 158, 163, 164, 165, 169, 170, 171, 173, 174, 178, 179, 180, 181, 182, 183, 187, 188, 189, 190, 195, 196, 198, 199, 202, 203, 204, 207, 210, 211, 212, 214, 215, 216, 217, 219, 220, 221, 222, 224, 225, 226, 227, 230, 234, 241, 242, 243, 244, 246, 247, 248, 249, 251, 253, 255, 257, 258, 260, 261, 262, 263, 264, 265, 267, 268, 269, 271, 272, 273, 274, 276, 277, 278, 280, 281, 282, 283, 286, 288, 292, 294, 298, 305, 306, 309, 310, 312, 313, 314, 315, 316, 319, 320, 323, 324, 325, 328, 329, 330, 331, 335, 336, 337, 338, 339, 341, 346, 350, 352, 353, 354, 357, 359, 360, 361, 362, 367, 368, 369, 373, 374, 376, 377, 378, 379, 381, 382, 